Code for baseline originating from: https://towardsdatascience.com/cook-your-first-u-net-in-pytorch-b3297a844cf3

In [1]:
import torch
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
from torch.nn.functional import relu
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from skimage import io
import torch
from skimage import color
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.init as init
from PIL import Image, ImageOps


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('y')
else:
    device = torch.device('cpu')
    print('n')


y


In [3]:
device = torch.device('cpu')

In [4]:
class FashionDataset(Dataset):
    def __init__(self, csv_file, img_dir, segm_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.segm_dir = segm_dir
        self.transform = transform


    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir,
                                self.data_frame.iloc[idx, 1])

        segm_name = os.path.join(self.segm_dir,
                                self.data_frame.iloc[idx, -1])

        image = Image.open(img_name)
        label = Image.open(segm_name)


        #Convert to grayscale 
        # image = image.convert('L')
        
        #Resize image and label to 572x572
        image = image.resize((572, 572))
        label = label.resize((572, 572))

        #Crop label to match probability output from model
        crop = transforms.CenterCrop(388)
        label = crop(label)

        
        #Normalize the image
        image = np.asarray(image)
        min_image = np.min(image)
        max_image = np.max(image)
        image = (image - min_image)/(max_image - min_image + 1e-4)
        
        #Convert to tensors
        image = torch.tensor(np.array(image)).float().permute(2, 0, 1)
        label = torch.tensor(np.array(label)).long()

        #sample = {'x': , 'y': }
        sample = {'x': image, 'y': label}
    
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [5]:
class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        # Encoder
        # In the encoder, convolutional layers with the Conv2d function are used to extract features from the input image. 
        # Each block in the encoder consists of two convolutional layers followed by a max-pooling layer, with the exception of the last block which does not include a max-pooling layer.
        # -------
        # input: 572x572x3
        self.e11 = nn.Conv2d(3, 64, kernel_size=3, padding=0) # output: 570x570x64
        self.e12 = nn.Conv2d(64, 64, kernel_size=3, padding=0) # output: 568x568x64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 284x284x64

        # input: 284x284x64
        self.e21 = nn.Conv2d(64, 128, kernel_size=3, padding=0) # output: 282x282x128
        self.e22 = nn.Conv2d(128, 128, kernel_size=3, padding=0) # output: 280x280x128
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 140x140x128

        # input: 140x140x128
        self.e31 = nn.Conv2d(128, 256, kernel_size=3, padding=0) # output: 138x138x256
        self.e32 = nn.Conv2d(256, 256, kernel_size=3, padding=0) # output: 136x136x256
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 68x68x256

        # input: 68x68x256
        self.e41 = nn.Conv2d(256, 512, kernel_size=3, padding=0) # output: 66x66x512
        self.e42 = nn.Conv2d(512, 512, kernel_size=3, padding=0) # output: 64x64x512
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 32x32x512

        # input: 32x32x512
        self.e51 = nn.Conv2d(512, 1024, kernel_size=3, padding=0) # output: 30x30x1024
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3, padding=0) # output: 28x28x1024


        # Decoder        

        self.centercrop1 = transforms.CenterCrop(56)
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3, padding=0)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3, padding=0)

        self.centercrop2 = transforms.CenterCrop(104)
        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3, padding=0)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3, padding=0)
        
        self.centercrop3 = transforms.CenterCrop(200)
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3, padding=0)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3, padding=0)

        self.centercrop4 = transforms.CenterCrop(392)
        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3, padding=0)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3, padding=0)

        # Output layer
        self.outconv = nn.Conv2d(64, n_class, kernel_size=1)


    def forward(self, x):
        # Encoder
        xe11 = relu(self.e11(x))
        xe12 = relu(self.e12(xe11))
        xp1 = self.pool1(xe12)

        xe21 = relu(self.e21(xp1))
        xe22 = relu(self.e22(xe21))
        xp2 = self.pool2(xe22)

        xe31 = relu(self.e31(xp2))
        xe32 = relu(self.e32(xe31))
        xp3 = self.pool3(xe32)

        xe41 = relu(self.e41(xp3))
        xe42 = relu(self.e42(xe41))
        xp4 = self.pool4(xe42)

        xe51 = relu(self.e51(xp4))
        xe52 = relu(self.e52(xe51))
       
        # Decoder
        xe42 = self.centercrop1(xe42) #ADDED
        xu1 = self.upconv1(xe52)
        xu11 = torch.cat([xu1, xe42], dim=1)
        xd11 = relu(self.d11(xu11))
        xd12 = relu(self.d12(xd11))

        xe32 = self.centercrop2(xe32) #ADDED
        xu2 = self.upconv2(xd12)
        xu22 = torch.cat([xu2, xe32], dim=1)
        xd21 = relu(self.d21(xu22))
        xd22 = relu(self.d22(xd21))

        xe22 = self.centercrop3(xe22) #ADDED        
        xu3 = self.upconv3(xd22)
        xu33 = torch.cat([xu3, xe22], dim=1)
        xd31 = relu(self.d31(xu33))
        xd32 = relu(self.d32(xd31))

        xe12 = self.centercrop4(xe12) #ADDED                
        xu4 = self.upconv4(xd32)
        xu44 = torch.cat([xu4, xe12], dim=1)
        xd41 = relu(self.d41(xu44))
        xd42 = relu(self.d42(xd41))

        # Output layer
        out = self.outconv(xd42)

        return out

In [6]:
def train_model(model, dataloader, optimizer, loss_fn, num_epochs, bs, verbose = False):
    loss_dict = {'train':[],'validate':[]}
    phases = ['train','validate']
    best_loss = 1e30
    since = time.time()
    for i in range(num_epochs):
        print('Epoch: {}/{}'.format(i, num_epochs-1))
        print('-'*10)
        for p in phases:
            # running_correct = 0
            running_loss = 0
            running_total = 0
            if p == 'train':
                model.train()
            else:
                model.eval()
            for data in dataloader[p]:
                optimizer.zero_grad()            
                image = data['x'].to(device)
                label = data['y'].to(device)
                output = model(image)
                print('hi')

                # output_flattened = output.permute(0, 2, 3, 1).contiguous().view(-1, 7)
                # label_flattened = label.view(-1).contiguous()  
                loss = loss_fn(output, label)     
                print('hi')

                reshaped_output = output.view(bs, 8, -1)
                _, preds = torch.max(reshaped_output, dim = 1)
                preds = preds.view(bs,388,388)
                num_imgs = image.size()[0]

                running_loss += loss.item()*num_imgs
                running_total += num_imgs
                
                if p== 'train':
                    loss.backward()
                    optimizer.step()
            epoch_loss = float(running_loss/running_total)
            if verbose or (i%10 == 0):
                print('Phase:{}, epoch loss: {:.4f}'.format(p, epoch_loss))

            loss_dict[p].append(epoch_loss)
            if p == 'validate':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = model.state_dict()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    model.load_state_dict(best_model_wts)
    
    return model, loss_dict




In [7]:
bs = 32

torch.cuda.empty_cache()

current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'data')

# Now you can reference files in the data directory
file_path = os.path.join(data_directory, 'your_file.txt')

dataset = {'train': FashionDataset(os.path.join(data_directory, 'train_sample.csv'),'/vast/amr10211/cv_data/images', '/vast/amr10211/cv_data/segm_preprocessed'), 
           'validate': FashionDataset(os.path.join(data_directory, 'val_sample.csv'),'/vast/amr10211/cv_data/images', '/vast/amr10211/cv_data/segm_preprocessed'), 
           'test': FashionDataset(os.path.join(data_directory, 'test_sample.csv'),'/vast/amr10211/cv_data/images', '/vast/amr10211/cv_data/segm_preprocessed')}
dataloader = {x: DataLoader(dataset[x], batch_size=bs,
                        shuffle=True, num_workers=0) for x in ['train', 'validate']}

model = UNet(n_class = 8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
model, loss_dict = train_model(model, dataloader, optimizer, loss_fn=criterion, num_epochs=10, bs=bs, verbose=True)


Epoch: 0/9
----------
hi
hi
